In [1]:
# Author: Fan Zhang
# 2018-10-16
# Deep Variational Canonical Correlation (VCCA-private)
# Modify Weiran Wang, 2017 code in single-cell and bulk
import numpy as np
import math
import os
import tensorflow as tf
import vcca_IM as vcca
from myreadinput import read_sc

In [2]:
# using argparse and this module is used to write user-friendly command-line interfaces, 
# so it seems, it has a conflict with Jupyter Notebook.
# import argparse
# parser=argparse.ArgumentParser()
# parser.add_argument("--Z", default=10, help="Dimensionality of features", type=int)
# parser.add_argument("--H1", default=10, help="Dimensionality of private variables for view 1", type=int) 
# parser.add_argument("--H2", default=10, help="Dimensionality of private variables for view 2", type=int)
# parser.add_argument("--IM", default=0.0, help="Regularization constant for the IM penalty", type=float)
# parser.add_argument("--dropprob", default=0.0, help="Dropout probability of networks.", type=float) 
# parser.add_argument("--checkpoint", default="./vcca_test", help="Path to saved models", type=str) 
# args=parser.parse_args()
from easydict import EasyDict as edict
args = edict({
    "Z": 10,
    "H1": 10,
    "H2": 10,
    "IM": 0.0,
    "dropprob": 0.0, # May try hier dropout: 0.3 with improved performance
    "checkpoint": "./vcca_res"
})
args

{'Z': 10,
 'H1': 10,
 'H2': 10,
 'IM': 0.0,
 'dropprob': 0.0,
 'checkpoint': './vcca_res'}

In [3]:
    # Set random seeds.
    np.random.seed(0)
    tf.set_random_seed(0)

In [4]:
    # Obtain parsed arguments.
    Z=args.Z
    print("Dimensionality of shared variables: %d" % Z)
    H1=args.H1
    print("Dimensionality of view 1 private variables: %d" % H1)
    H2=args.H2
    print("Dimensionality of view 2 private variables: %d" % H2)
    IM_penalty=args.IM
    print("Regularization constant for IM penalty: %f" % IM_penalty)
    dropprob=args.dropprob
    print("Dropout rate: %f" % dropprob)
    checkpoint=args.checkpoint
    print("Trained model will be saved at %s" % checkpoint)

Dimensionality of shared variables: 10
Dimensionality of view 1 private variables: 10
Dimensionality of view 2 private variables: 10
Regularization constant for IM penalty: 0.000000
Dropout rate: 0.000000
Trained model will be saved at ./vcca_res


In [6]:
    # Some other configurations parameters
    losstype1=0
    losstype2=1
    learning_rate=0.0001
    l2_penalty=0
    latent_penalty=1.0

In [7]:
    # Define network architectures.
    # Replace 784 with the number of genes of inputs
    network_architecture=dict(
        # n_input1=784, 
        # n_input2=784, 
        n_input1=724, 
        n_input2=724, 
        n_z=Z,  # Dimensionality of shared latent space
        # F_hidden_widths=[1024, 1024, 1024, Z],
        F_hidden_widths=[1024, 1024, 1024, Z],
        F_hidden_activations=[tf.nn.relu, tf.nn.relu, tf.nn.relu, None],
        n_h1=H1, # Dimensionality of individual latent space of view 1
        G1_hidden_widths=[1024, 1024, 1024, H1],
        G1_hidden_activations=[tf.nn.relu, tf.nn.relu, tf.nn.relu, None],
        n_h2=H2, # Dimensionality of individual latent space of view 2
        G2_hidden_widths=[1024, 1024, 1024, H2],
        G2_hidden_activations=[tf.nn.relu, tf.nn.relu, tf.nn.relu, None],
        # H1_hidden_widths=[1024, 1024, 1024, 784],
        H1_hidden_widths=[1024, 1024, 1024, 724],
        H1_hidden_activations=[tf.nn.relu, tf.nn.relu, tf.nn.relu, tf.nn.sigmoid],
        # H2_hidden_widths=[1024, 1024, 1024, 784],
        H2_hidden_widths=[1024, 1024, 1024, 724],
        H2_hidden_activations=[tf.nn.relu, tf.nn.relu, tf.nn.relu, tf.nn.sigmoid]
        )

In [8]:
    # First, build the model.
    model=vcca.VCCA(network_architecture, losstype1, losstype2, learning_rate, l2_penalty, latent_penalty)
    saver=tf.train.Saver()

Building view 1 recognition network F ...
	Layer 1 ...
	Layer 2 ...
	Layer 3 ...
	Layer 4 ...
Building view 1 private network G1 ...
	Layer 1 ...
	Layer 2 ...
	Layer 3 ...
	Layer 4 ...
Building view 2 private network G2 ...
	Layer 1 ...
	Layer 2 ...
	Layer 3 ...
	Layer 4 ...
Building view 1 reconstruction network H1 ...
	Layer 1 ...
	Layer 2 ...
	Layer 3 ...
	Layer 4 ...
Building view 2 reconstruction network H2 ...
	Layer 1 ...
	Layer 2 ...
	Layer 3 ...
	Layer 4 ...
Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [9]:
    # Second, load the saved moded, if provided.
    if checkpoint and os.path.isfile(checkpoint):
        print("loading model from %s " % checkpoint)
        saver.restore(model.sess, checkpoint)
        epoch=model.sess.run(model.epoch)
        print("picking up from epoch %d " % epoch)
        tunecost=model.sess.run(model.tunecost)
        print("tuning cost so far:")
        print(tunecost[0:epoch])
    else:
        print("checkpoint file not given or not existent!")

checkpoint file not given or not existent!


In [10]:
    # File for saving classification results.
    classfile=checkpoint + '_classify.mat'
    if os.path.isfile(classfile):
        print("Job is already finished!")
        return

SyntaxError: 'return' outside function (<ipython-input-10-97465b486ccb>, line 5)

In [11]:
    # Third, load the data.
    from scvi.dataset import LoomDataset, CsvDataset, Dataset10X, AnnDataset
    data1_raw = CsvDataset("sc_724rows_5265cols.csv.gz", save_path='./', new_n_genes=724, compression='gzip')
    data2_raw = CsvDataset("bulk_724rows_167cols.csv.gz", save_path='./', new_n_genes=724, compression='gzip')
    print(np.shape(data1_raw.X))
    print(np.shape(data2_raw.X))

File ./sc_724rows_5265cols.csv.gz already downloaded
Preprocessing dataset
Finished preprocessing dataset
File ./bulk_724rows_167cols.csv.gz already downloaded
Preprocessing dataset
Finished preprocessing dataset
(5265, 724)
(167, 724)


In [12]:
# Import meta data
import pandas as pd
%matplotlib inline
sc_meta = pd.read_table("meta_sc_724rows_5265cols.tsv")
bulk_meta = pd.read_table("meta_bulk_724rows_167cols.tsv")
print(np.shape(sc_meta))
print(np.shape(bulk_meta))

(5265, 41)
(167, 87)


In [17]:
# Extend the dataset with less samples to the same number of samples of the other dataset
N = np.int64(np.floor(np.shape(data1_raw.X)[0]/np.shape(data2_raw.X)[0]))
test = np.vstack([data2_raw.X]* N)
# print(np.shape(data1_raw.X)[0] - np.shape(test)[0])
b = data2_raw.X[:(np.shape(data1_raw.X)[0] - np.shape(test)[0]),:]
test = np.concatenate((test, b), axis=0)
print(np.shape(test))
# Shuffle the rows
data2_extend = np.take(test, np.random.permutation(test.shape[0]),axis=0,out=test);
print(np.shape(data2_extend))

(5265, 724)
(5265, 724)


In [18]:
def load_single_celldata(ratio_train, dataset, meta):
#     print(np.shape(dataset))
    df_matrix = np.array(dataset)
#     df_matrix_t = df_matrix.transpose()
#     print(np.shape(df_matrix_t))
    N, D = df_matrix.shape
    #Split the training data (training and validation) and test dataset.
    ind_cut = int(ratio_train * N)
    ind = np.random.permutation(N)
    
    train_val_data = df_matrix[ind[:ind_cut], :]
    test = df_matrix[ind[ind_cut:], :]
    meta_train_val = np.array(meta)[ind[:ind_cut],:]
    meta_test = np.array(meta)[ind[ind_cut:],:]
    
    #Split the training and validation dataset.
    N, D = train_val_data.shape
    ind_cut = int(ratio_train * N)
    ind = np.random.permutation(N)
    
    #Input data
    train = train_val_data[ind[:ind_cut], :]
    tune = train_val_data[ind[ind_cut:], :]
    meta_train = meta_train_val[ind[:ind_cut], :]
    meta_tune = meta_train_val[ind[ind_cut:], :]

    return train, tune, test, meta_train, meta_tune, meta_test

def load_bulkdata(ratio_train, dataset):
#     print(np.shape(dataset))
    df_matrix = np.array(dataset)
#     df_matrix_t = df_matrix.transpose()
#     print(np.shape(df_matrix_t))
    N, D = df_matrix.shape
    #Split the training data (training and validation) and test dataset.
    ind_cut = int(ratio_train * N)
    ind = np.random.permutation(N)
    
    train_val_data = df_matrix[ind[:ind_cut], :]
    test = df_matrix[ind[ind_cut:], :]
    
    #Split the training and validation dataset.
    N, D = train_val_data.shape
    ind_cut = int(ratio_train * N)
    ind = np.random.permutation(N)
    
    #Input data
    train = train_val_data[ind[:ind_cut], :]
    tune = train_val_data[ind[ind_cut:], :]

    return train, tune, test

trainData1,tuneData1,testData1,meta_trainData1,meta_tuneData1,meta_testData1= load_single_celldata(ratio_train = 0.8, dataset = data1_raw.X, meta = sc_meta)
trainData2,tuneData2,testData2= load_bulkdata(ratio_train = 0.8, dataset = data2_extend)

# meta_trainData1[:10,1:10]
print(np.shape(meta_trainData1))
print(np.shape(meta_tuneData1))
print(np.shape(meta_testData1))

print(np.shape(trainData1))
print(np.shape(trainData2))

print(np.shape(testData1))
print(np.shape(testData2))

(3369, 41)
(843, 41)
(1053, 41)
(3369, 724)
(3369, 724)
(1053, 724)
(1053, 724)


In [19]:
trainLabel = np.full(np.shape(trainData1)[0], "0")
print(np.shape(trainLabel))

tuneLabel = np.full(np.shape(tuneData1)[0], "1")
print(np.shape(tuneLabel))

testLabel = np.full(np.shape(testData1)[0], "2")
print(np.shape(testLabel))

(3369,)
(843,)
(1053,)


In [20]:
print(trainData1.shape[0])
print(trainLabel.shape[0])
print(trainData2.shape[0])
trainData1.shape[0] == trainLabel.shape[0]
trainData2.shape[0] == trainLabel.shape[0]

tuneData1.shape[0] == tuneLabel.shape[0]
tuneData2.shape[0] == tuneLabel.shape[0]

testData1.shape[0] == testLabel.shape[0]
testData2.shape[0] == testLabel.shape[0]

3369
3369
3369


True

In [21]:
trainData,tuneData,testData=read_sc(trainData1,trainData2,trainLabel,tuneData1,tuneData2,tuneLabel,testData1,testData2,testLabel)

type conversion view 1
type conversion view 2
type conversion view 1
type conversion view 2
type conversion view 1
type conversion view 2


In [22]:
trainData.num_examples

3369

In [23]:
# Traning.
# batch_size = 64, 128, 256, 512
# max_epochs = 50, 100, 150, 200
# dropprob = 0.0, 0.1, 0.2, 0.3
model=vcca.train(model, trainData, tuneData, saver, checkpoint, batch_size=52, max_epochs=50, save_interval=20, keepprob=(1.0-dropprob))

Current learning rate 0.000100
Epoch: 0001, train regret=-4859.82466442, tune cost=-10805.88281250
latent_loss=196.47549438, nll1=-12943.04199219, nll2=1940.68383789
Current learning rate 0.000100
Epoch: 0002, train regret=-11238.72505635, tune cost=-11222.63769531
latent_loss=141.02593994, nll1=-13282.30468750, nll2=1918.64038086
Current learning rate 0.000100
Epoch: 0003, train regret=-11520.87269034, tune cost=-11586.35839844
latent_loss=98.11020660, nll1=-13549.32812500, nll2=1864.85913086
Current learning rate 0.000100
Epoch: 0004, train regret=-11855.02445435, tune cost=-11750.96582031
latent_loss=69.35379028, nll1=-13657.86621094, nll2=1837.54650879
Current learning rate 0.000100
Epoch: 0005, train regret=-12012.25616721, tune cost=-11826.46679688
latent_loss=62.54430389, nll1=-13697.68261719, nll2=1808.67175293
Current learning rate 0.000100
Epoch: 0006, train regret=-11999.02705620, tune cost=-11882.30859375
latent_loss=53.87675476, nll1=-13724.59960938, nll2=1788.41381836
Cur

Epoch: 0050, train regret=-12726.69746795, tune cost=-12306.11816406
latent_loss=41.84127426, nll1=-14076.74218750, nll2=1728.78283691


In [24]:
# Plot lower bound of objective function
model.cost

<tf.Tensor 'add_16:0' shape=() dtype=float32>

In [25]:
# Visualize z
print("Visualizing shared variables!")
trainData,tuneData,testData=read_sc(trainData1,trainData2,trainLabel,tuneData1,tuneData2,tuneLabel,testData1,testData2,testLabel)
z_train, _=model.transform_shared(1, trainData.images1)
z_tune, _=model.transform_shared(1, tuneData.images1)
z_test, _=model.transform_shared(1, testData.images1)

In [26]:
# Run tSNE on the z_test
from sklearn.manifold import TSNE
tsne=TSNE(perplexity=20, n_components=2, init="pca", n_iter=3000)
z_tsne=tsne.fit_transform(np.asfarray(z_test[::2], dtype="float") )

In [27]:
if H1>0:
        print("Visualizing private variables!")
        h1_test, _= model.transform_private(1, testData.images1)
        h1_train, _= model.transform_private(1, trainData.images1)
        tsne=TSNE(perplexity=20, n_components=2, init="pca", n_iter=3000)
        h1_tsne=tsne.fit_transform( np.asfarray(h1_test[::2], dtype="float") )
else:
        h1_test=-1
        h1_tsne=-1

Visualizing private variables!


In [28]:
if H2>0:
        print("Visualizing private variables!")
        h2_test, _= model.transform_private(1, testData.images2)
        h2_train, _= model.transform_private(1, trainData.images2)
        tsne=TSNE(perplexity=20, n_components=2, init="pca", n_iter=3000)
        h2_tsne=tsne.fit_transform( np.asfarray(h2_test[::2], dtype="float") )
else:
        h2_test=-1
        h2_tsne=-1

Visualizing private variables!


In [29]:
print(np.shape(z_train))
print(np.shape(z_tsne))
print(np.shape(z_tune))
print(np.shape(z_test))
print(np.shape(h1_test))
print(np.shape(h1_tsne))
print(np.shape(h2_test))
print(np.shape(h2_tsne))
print(np.shape(h1_train))
print(np.shape(h2_train))

(3369, 10)
(527, 2)
(843, 10)
(1053, 10)
(1053, 10)
(527, 2)
(1053, 10)
(527, 2)
(3369, 10)
(3369, 10)


In [30]:
import scipy.io as sio
sio.savemat(classfile, {
        # 'tuneerr':best_error_tune,  'testerr':best_error_test,
        'z_train':z_train,  'z_tune':z_tune,  'z_test':z_test,  'z_tsne':z_tsne,
        'h1_test':h1_test,  'h1_tsne':h1_tsne,  'h2_test':h2_test,  'h2_tsne':h2_tsne,
        'meta_trainData1':meta_trainData1[:,1],  'meta_tuneData1':meta_tuneData1[:,1],  'meta_testData1':meta_testData1[:,1]
        })

In [31]:
# print(np.shape(meta_trainData1))
# meta_trainData1[:,1]